# DESI PV Survey Target Matches

Grab the list of targets from the DESI PV survey that are available in some spectroscopic reduction, which can be specified by the user.

The list of PV secondary targets is provided in a set of FITS files produced by [Khaled Said](mailto:k.saidahmedsoliman@uq.edu.au) in the folder `/global/homes/k/ksaid/desi_pv/savepath_dr9_corr` at NERSC.

DESI observations are taken from the [redshift database](https://desi.lbl.gov/trac/wiki/DESIDatabase) maintained at NERSC by Rob Knop.

In [1]:
from desispec.io import read_spectra
from desispec.coaddition import coadd_cameras
from desispec.interpolation import resample_flux
from desispec.resolution import Resolution

import redrock.templates

rrtemplates = dict()
for filename in redrock.templates.find_templates():
    t = redrock.templates.Template(filename)
    rrtemplates[(t.template_type, t.sub_type)] = t

from astropy import units as u
from astropy.table import Table, vstack, hstack
from astropy.coordinates import SkyCoord, match_coordinates_sky
from astropy.time import Time
from astropy.wcs import WCS
from astropy.visualization.wcsaxes import SphericalCircle

from datetime import datetime, timedelta

from scipy.ndimage import gaussian_filter1d

import os
from glob import glob

import psycopg2

from tqdm.notebook import tqdm_notebook

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

DEBUG: Read templates from /global/common/software/desi/cori/desiconda/20211217-2.0.0/code/redrock-templates/master
DEBUG: Using default redshift range -0.0050-1.6997 for rrtemplate-galaxy.fits
DEBUG: Using default redshift range 0.0500-5.9934 for rrtemplate-qso.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-A.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-B.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-CV.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-F.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-G.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-K.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-M.fits
DEBUG: Using default redshift range -0.0020-0.0020 for rrtemplate-star-WD.fits


In [2]:
mpl.rc('font', size=14)
mpl.rc('figure', max_open_warning = 0)

## Database Matching Functions

Given a target table and spectroscopic reduction, match all observed targets using the DESI redshift database.

In [35]:
def match_targets(pvtargtab, redux='daily', search='healpix'):
    """Match PV targets against the redshift DB for a particular spectroscopic reduction.
    
    Parameters
    ----------
    pvtargtab : astropy.Table
        Table of PV target info. Specifically need the RA, DEC, PVTYPE, and SGA_ID fields.
    redux : str
        Spectroscopic reduction: e.g., 'daily', 'everest', 'fuji', 'guadalupe', ...
    search : str
        'healpix' to search the HEALPix tables, 'tiles' to search the tiles tables.
        
    Returns
    -------
    desi_targets : astropy.Table
        Joined table of DESI redshifts and PV targets for all matches.
    """
    # Accumulate data in this table.
    desi_targets = None
        
    try:
        db = psycopg2.connect(host='decatdb.lbl.gov', database='desidb', user='desi')
        cursor = db.cursor()
        # cursor.execute('SET search_path TO da, public;')

        # Loop over all TNS alerts and perform a coordinate match with DESI observations.
        N = len(pvtargtab)
        n = 0
        with tqdm_notebook(total=N) as progress_bar:

            for i, obj in enumerate(pvtargtab):
                ra, dec = obj['RA'], obj['DEC']

                # Enable search in HEALPix tables.
                if search == 'healpix':
                    query = 'SELECT f.targetid,f.target_ra,f.target_dec,h.healpix,h.survey,r.z,r.zerr,r.zwarn,r.deltachi2,h.filename\n' \
                            f'FROM {redux}.healpix_fibermap f\n' \
                            f'INNER JOIN {redux}.healpix h ON f.healpix_id=h.id\n' \
                            f'INNER JOIN {redux}.healpix_redshifts r ON r.healpix_id=h.id AND r.targetid=f.targetid\n' \
                            f'WHERE q3c_radial_query( f.target_ra, f.target_dec, {ra}, {dec}, 1./3600. );'
                    
                    colnames = ['TARGETID', 'TARGET_RA', 'TARGET_DEC', 'HEALPIX', 'SURVEY', 'Z', 'ZERR', 'ZWARN', 'DELTACHI2', 'FILENAME']
                # Enable search in tiles tables.
                elif search == 'tiles':
                    query = 'SELECT f.targetid,f.target_ra,f.target_dec,c.tileid,c.night,r.z,r.zerr,r.zwarn,r.deltachi2,c.filename\n' \
                            f'FROM {redux}.tiles_fibermap f\n' \
                            f'INNER JOIN {redux}.cumulative_tiles c ON f.cumultile_id=c.id\n' \
                            f'INNER JOIN {redux}.tiles_redshifts r ON r.cumultile_id=c.id AND r.targetid=f.targetid\n' \
                            f'WHERE q3c_radial_query( f.target_ra, f.target_dec, {ra}, {dec}, 1./3600. );'
                    colnames = ['TARGETID', 'TARGET_RA', 'TARGET_DEC', 'TILEID', 'NIGHT', 'Z', 'ZERR', 'ZWARN', 'DELTACHI2', 'FILENAME']
                else:
                    raise ValueError(f'Search {search} not recognized; use "healpix" or "tiles."')

                cursor.execute(query)
                rows = cursor.fetchall()

                if rows:
                    # Convert postgresql row output to an astropy Table.
                    data = Table(list(map(list, zip(*rows))),
                                 names=colnames)

                    # hstack the postgresql rows with the PV target info.
                    # The following vstack loop ensures every row gets a match.
                    pv_data = obj
                    if len(data) > 1:
                        for j in range(1, len(data)):
                            pv_data = vstack([pv_data, obj])
                    data = hstack([data, pv_data['PVTYPE', 'SGA_ID', 'RA', 'DEC']])

                    # Accumulate matched targets.
                    if desi_targets is None:
                        desi_targets = data
                    else:
                        desi_targets = vstack([desi_targets, data], join_type='outer')

                if (i+1) % 100 == 0:
                    progress_bar.update(100)
                    n += 100

            if n < N:
                progress_bar.update(N - n)

    except (Exception, psycopg2.Error) as error:
        print(error)
    finally:
        if db is not None:
            db.close()
            
    return desi_targets

## Perform Coordinate Match between DESI Observations and the PV Target Files

Match PV observations within 1" of a DESI fiber.

Read FITS files of PV targets. In this notebook, we only download targets relevant to the DESI Tully-Fisher sample, which corresponds to these three files:
* `pv_tf.fits`: targets on the semi-major axes of spiral galaxies, selected for the Tully-Fisher analysis.
* `pv_ext.fits`: other targets on spatially extended spiral galaxies (e.g., HII regions).
* `pv_sga.fits`: centers of SGA galaxies.

In [32]:
pv_ext = Table.read('/global/homes/k/ksaid/desi_pv/savepath_dr9_corr/pv_ext.fits', hdu=1)
pv_sga = Table.read('/global/homes/k/ksaid/desi_pv/savepath_dr9_corr/pv_sga.fits', hdu=1)
pv_tf = Table.read('/global/homes/k/ksaid/desi_pv/savepath_dr9_corr/pv_tf.fits', hdu=1)

### Process fuji Observations

In [30]:
pv_ext_fuji = match_targets(pv_ext, redux='fuji')
pv_ext_fuji

  0%|          | 0/20486 [00:00<?, ?it/s]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC
int64,float64,float64,int64,str3,float64,float64,int64,float64,str63,bytes3,int64,float64,float64
1092899764699136,216.00143660376,34.881062410803,10515,sv3,0.512530942294242,0.0001125096994758,4,5.75281726941466,fuji/healpix/sv3/dark/105/10515/redrock-sv3-dark-10515.fits,EXT,682254,216.0014366037598,34.881062410803
1079700659109889,216.061747702864,34.8365760573346,10515,sv3,1.5277305869098,0.000248864600940801,4,8.71738664060831,fuji/healpix/sv3/bright/105/10515/redrock-sv3-bright-10515.fits,EXT,682254,216.06174770286376,34.836576057334575
1092894798643201,216.061747702864,34.8365760573346,10515,sv3,1.01290900857488,6.38649008792103e-05,4,2.01506862044334,fuji/healpix/sv3/dark/105/10515/redrock-sv3-dark-10515.fits,EXT,682254,216.06174770286376,34.836576057334575
1079037296377859,208.975477166485,5.30336633406908,26091,sv3,1.49877155103453,0.00022479094539867,0,11.9354891255498,fuji/healpix/sv3/bright/260/26091/redrock-sv3-bright-26091.fits,EXT,865490,208.9754771664854,5.303366334069084
1092231435911171,208.975477166485,5.30336633406908,26091,sv3,1.18699867849368,0.000116017737064761,4,1.74350547604263,fuji/healpix/sv3/dark/260/26091/redrock-sv3-dark-26091.fits,EXT,865490,208.9754771664854,5.303366334069084
1079037300572160,209.084716493457,5.20650793636111,26091,sv3,0.907146237131769,0.000133516273090146,0,11.1716398671269,fuji/healpix/sv3/bright/260/26091/redrock-sv3-bright-26091.fits,EXT,865490,209.0847164934571,5.206507936361107
1092231440105472,209.084716493457,5.20650793636111,26091,sv3,1.55395322376555,0.000108078787702566,4,4.13443431630731,fuji/healpix/sv3/dark/260/26091/redrock-sv3-dark-26091.fits,EXT,865490,209.0847164934571,5.206507936361107
1078989217071107,217.423332498867,3.20319258782244,26065,sv3,0.680248512223176,8.41292468167563e-05,4,0.725679285824299,fuji/healpix/sv3/bright/260/26065/redrock-sv3-bright-26065.fits,EXT,1413430,217.42333249886704,3.203192587822438
1092183356604419,217.423332498867,3.20319258782244,26065,sv3,0.464078931593149,6.04317367217602e-05,4,5.38805882632732,fuji/healpix/sv3/dark/260/26065/redrock-sv3-dark-26065.fits,EXT,1413430,217.42333249886704,3.203192587822438


In [33]:
pv_sga_fuji = match_targets(pv_sga, redux='fuji')
pv_sga_fuji

  0%|          | 0/81611 [00:00<?, ?it/s]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC
int64,float64,float64,int64,str3,float64,float64,int64,float64,str63,bytes3,int64,float64,float64
2305843022014712506,197.156461215073,79.7870276386574,12210,sv1,0.0353719025788651,8.63955563819646e-07,0,215493.827211023,fuji/healpix/sv1/backup/122/12210/redrock-sv1-backup-12210.fits,SGA,588991,197.1565928926111,79.78701060360044
39633244072708111,172.478975482847,49.8111296555183,5613,sv2,0.212886036612501,2.56065525084539e-05,0,9280.4979827255,fuji/healpix/sv2/bright/56/5613/redrock-sv2-bright-5613.fits,SGA,462542,172.4789754828471,49.81112965551834
2305843015098307568,172.88665773677,49.8575168421019,5613,sv2,0.0686768543072607,3.64798748181209e-05,0,540.748181710485,fuji/healpix/sv2/backup/56/5613/redrock-sv2-backup-5613.fits,SGA,181737,172.88664359320452,49.85749695685062
39633247981799461,172.488634263549,49.9945914090987,5613,sv2,0.0464544408476083,9.59760084658303e-06,0,20088.77623064,fuji/healpix/sv2/bright/56/5613/redrock-sv2-bright-5613.fits,SGA,105777,172.4886342635487,49.99459140909872
39633247977604638,172.046969027532,49.8765652794558,5613,sv2,0.032810246228186,1.33863281529401e-06,0,25134.5055181811,fuji/healpix/sv2/bright/56/5613/redrock-sv2-bright-5613.fits,SGA,205849,172.0469690275316,49.87656527945583
39633240146840330,172.621403181147,49.5827213857747,5613,sv2,0.0957683291848579,2.9732896882583e-06,0,567732.339778543,fuji/healpix/sv2/bright/56/5613/redrock-sv2-bright-5613.fits,SGA,291430,172.6214031811471,49.58272138577475
39633240155229162,173.423295043232,49.5987878330557,5618,sv2,0.0102086746614462,5.04173620818413e-06,0,937.169478463009,fuji/healpix/sv2/bright/56/5618/redrock-sv2-bright-5618.fits,SGA,929745,173.42329504323192,49.59878783305574
39633247994381773,173.539574562025,49.9247396412793,5624,sv2,0.167507289179424,2.5272786938781e-05,0,1592.93692483939,fuji/healpix/sv2/bright/56/5624/redrock-sv2-bright-5624.fits,SGA,377047,173.53957456202534,49.92473964127928
39633240163615366,173.93563451952,49.5372757009344,5618,sv2,0.167551024270997,2.94086125594955e-06,0,20498.8166685104,fuji/healpix/sv2/bright/56/5618/redrock-sv2-bright-5618.fits,SGA,1281727,173.93563451951957,49.53727570093442


In [ ]:
pv_tf_fuji = match_targets(pv_tf, redux='fuji')
pv_tf_fuji

  0%|          | 0/389316 [00:00<?, ?it/s]

In [ ]:
isgoodz = (pv_tf_fuji['ZWARN']==0) & (pv_tf_fuji['DELTACHI2']>25)
pv_tf_fuji[isgoodz]

#### Dump Output to a File

Stack the `ext`, `sga`, and `tf` tables and save the output.

In [ ]:
pv_fuji = vstack([pv_ext_fuji, pv_sga_fuji, pv_tf_fuji])
pv_fuji.write('desi_pv_tf_fuji.fits', overwrite=True)
# desi_targets.write('desi_tns_matches_thru_202206.fits', overwrite=True)

### Process guadalupe Observations